In [ ]:
%%capture
!pip install unsloth vllm
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install git+https://github.com/huggingface/transformers.git

In [ ]:
!pip install bitsandbytes

In [ ]:
pip install unsloth-zoo

In [ ]:
import os
# os.environ['TRITON_DISABLE_LINE_INFO'] = '1' # Optional, tested with and without
os.environ['TRITON_JIT_DISABLE_OPT'] = '1' # Likely the most critical change

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from unsloth import FastLanguageModel

model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit"  # or 3.2, 70B, Instruct, etc.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=1024,
    load_in_4bit=True,  # use QLoRA
    dtype=None,
)


In [ ]:
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

In [ ]:
!pip uninstall -y protobuf sentencepiece
!pip install protobuf==3.20.3 sentencepiece --force-reinstall

In [ ]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=16,
    lora_dropout=0.0,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    bias="none",
    use_gradient_checkpointing = False,
    use_rslora=False,
)
tokenizer = get_chat_template(tokenizer, chat_template="chatml")

Unsloth 2025.6.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.
Unsloth: Will map <|im_end|> to EOS = <|end_of_text|>.


In [ ]:
import json
from datasets import Dataset

# Load your JSON dataset
with open("/content/drive/MyDrive/combined_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Convert to chat format
def format_example(entry):
    user_prompt = (
        f"Question: {entry['question']}\n"
        f"Rubric: {entry['rubric']}\n"
        f"Ideal Answer: {entry['ideal_answer']}\n"
        f"Student Answer: {entry['student_answer']}\n"
        f"Strategy: {entry['strategy']}"
    )
    assistant_response = f"Score: {entry['score']}\nReview: {entry['cot_review']}"
    return {
        "user": user_prompt,
        "assistant": assistant_response
    }

# Apply transformation
chat_data = [format_example(entry) for entry in data]
ds = Dataset.from_list(chat_data)


In [ ]:
ds[0]

{'user': "Question: Choose three principles from the list provided and explain how they can be applied to improve the agile software development process. Provide examples for each principle.\nRubric: Demonstration of 'Learn From History' principle (2 marks)\nApplication of 'Less Code Is Better Code' principle (2 marks)\nExplanation of 'Make The System Usable' principle (2 marks)\nExamples provided for each principle (2 marks)\nWriting style and organization (2 marks)\nIdeal Answer: The response demonstrates a strong understanding of the 'Learn From History' principle, highlighting the importance of building on past experiences to make informed decisions in the agile development process. The answer also illustrates the value of 'Less Code Is Better Code', providing examples of how concise and clear code can lead to a more maintainable and efficient system. Furthermore, the response emphasizes the significance of 'Make The System Usable', discussing the importance of user-friendly interf

In [ ]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template

# Get tokenizer (assume already loaded)
tokenizer = get_chat_template(tokenizer, chat_template="chatml")

# Apply formatting to dataset
ds = ds.map(lambda example: {
    "text": tokenizer.apply_chat_template(
        [{"role": "user", "content": example["user"]},
         {"role": "assistant", "content": example["assistant"]}],
        tokenize=False
    )
})


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./finetuned",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    fp16 = False,
    num_train_epochs=1,
    learning_rate=3e-5,
    save_total_limit=2,
    logging_steps=10,
      # optional
    max_steps=250,
)


def formatting_func(examples):
    return {"text": examples["text"]}

trainer = SFTTrainer(
    model=model,
    train_dataset=ds, # Use the processed dataset `ds`
    args=training_args,
    tokenizer=tokenizer,
    formatting_func=formatting_func, # Add the formatting function
)


In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 911 | Num Epochs = 3 | Total steps = 250
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: da23bce1656 (da23bce1656-vellore-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.335700
20,1.196700
30,1.054500
40,0.940800
50,0.875900
60,0.820900
70,0.803800
80,0.773500
90,0.745700
100,0.691300


wandb: WARNING The get_url method is deprecated and will be removed in a future release. Please use `run.url` instead.


TrainOutput(global_step=250, training_loss=0.6966035194396972, metrics={'train_runtime': 5418.5392, 'train_samples_per_second': 0.369, 'train_steps_per_second': 0.046, 'total_flos': 6.976765250047181e+16, 'train_loss': 0.6966035194396972})

In [ ]:
trainer.save_model("/content/drive/MyDrive/fine-tuned-model1")

In [ ]:
model.save_pretrained("llama3-grader")
tokenizer.save_pretrained("llama3-grader")


('llama3-grader/tokenizer_config.json',
 'llama3-grader/special_tokens_map.json',
 'llama3-grader/chat_template.jinja',
 'llama3-grader/tokenizer.json')

In [ ]:
chat = [
    {"role": "user", "content": f"""
"Question": "What is the primary objective of requirements modeling, and how does it contribute to the software development process?",
    "Rubric": "A clear description of the primary objective of requirements modeling, including its role in describing customer requirements (3 marks)\nAn explanation of how requirements modeling establishes a basis for software design, including the importance of bridging the gap between system-level representation and software design (4 marks)\nA discussion of the role of requirements modeling in defining a set of requirements that can be validated once the software is built, including the significance of validation in the software development process (3 marks)",
    "Ideal Answer": "The primary objective of requirements modeling is to create a variety of representations that describe what the customer requires, establishing a basis for the creation of a software design, and defining a set of requirements that can be validated once the software is built. Requirements modeling plays a crucial role in bridging the gap between a system-level representation that describes overall system and business functionality and a software design. By doing so, it enables the development of a software design that meets the customer's requirements and ensures that the software functions as intended. Furthermore, requirements modeling defines a set of requirements that can be validated once the software is built, allowing for the verification of the software's functionality and performance. This validation is essential in ensuring that the software meets the customer's needs and is free from defects, ultimately contributing to the success of the software development process.",
    "Student Answer": "To be honest, I'm not entirely sure about the primary objective of requirements modeling, but from what I've gathered, it's basically about creating a bunch of diagrams and documents that describe what the customer wants. I think it's supposed to help us understand the customer's requirements, but I'm not really clear on how it does that. \n\nI've heard that requirements modeling is important for software design, but I don't really get how it bridges the gap between system-level representation and software design. I mean, isn't software design just about writing code? I'm not sure how requirements modeling fits into that. Maybe it's like, a way to make sure we're building the right thing, but I'm not really sure.\n\nAs for validation, I think requirements modeling is supposed to help us define a set of requirements that we can check against once the software is built. But, to be honest, I'm not really sure how that works or why it's important. I know validation is a thing, but I don't really understand its significance in the software development process. Can we come back to this later? I feel like I'm missing something. \n\nOh, and I think requirements modeling might also have something to do with, like, making sure the software is usable and stuff. But, yeah, that's about it. I'm pretty lost on this topic, sorry.",
    "Strategy": "clarification"
     Please evaluate the student's answer by assigning a score from 0 to 10 and providing a concise review.
Return only this format:
Score: <score from 0 to 10>
Review: <your feedback>
"""}
]

response = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(response, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=150)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

LlamaForCausalLM has no `_prepare_4d_causal_attention_mask_with_cache_position` method defined in its base modeling class. Compiled forward passes will be sub-optimal. If you're writing code, see Llama for an example implementation. If you're a user, please report this issue on GitHub.


<|im_start|>user

"Question": "What is the primary objective of requirements modeling, and how does it contribute to the software development process?",
    "Rubric": "A clear description of the primary objective of requirements modeling, including its role in describing customer requirements (3 marks)
An explanation of how requirements modeling establishes a basis for software design, including the importance of bridging the gap between system-level representation and software design (4 marks)
A discussion of the role of requirements modeling in defining a set of requirements that can be validated once the software is built, including the significance of validation in the software development process (3 marks)",
    "Ideal Answer": "The primary objective of requirements modeling is to create a variety of representations that describe what the customer requires, establishing a basis for the creation of a software design, and defining a set of requirements that can be validated once the

In [ ]:
chat = [
    {"role": "user", "content": f"""
"Question": "What is dynamic programming and why is it useful in algorithm design?",
"Rubric": "A clear definition of dynamic programming, including its use of overlapping subproblems and optimal substructure (3 marks)\nAn explanation of how it differs from recursion and its efficiency benefits (4 marks)\nA discussion of its usefulness in solving problems like Fibonacci, knapsack, and matrix chain multiplication (3 marks)",
"Ideal Answer": "Dynamic programming (DP) is an algorithmic technique used to solve problems by breaking them down into simpler subproblems and storing their results to avoid redundant computations. It is applicable to problems with overlapping subproblems and optimal substructure. Unlike plain recursion, DP uses memoization or tabulation to improve efficiency, reducing time complexity significantly. For example, calculating Fibonacci numbers using recursion is exponential, while using DP makes it linear. DP is particularly useful in solving optimization problems like the 0/1 Knapsack, matrix chain multiplication, and longest common subsequence.",
"Student Answer": "Dynamic programming is like recursion but faster. It stores values so we don't repeat work. It’s used in problems like knapsack, I think. I'm not sure what optimal substructure means, but it helps in improving performance. I always get confused between top-down and bottom-up, but I know it’s useful.",
"Strategy": "clarification"
Please evaluate the student's answer by assigning a score from 0 to 10 and providing a concise review.
Return only this format:
Score: <score from 0 to 10>
Review: <your feedback>
"""}
]

response = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(response, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=150)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<|im_start|>user

"Question": "What is dynamic programming and why is it useful in algorithm design?",
"Rubric": "A clear definition of dynamic programming, including its use of overlapping subproblems and optimal substructure (3 marks)
An explanation of how it differs from recursion and its efficiency benefits (4 marks)
A discussion of its usefulness in solving problems like Fibonacci, knapsack, and matrix chain multiplication (3 marks)",
"Ideal Answer": "Dynamic programming (DP) is an algorithmic technique used to solve problems by breaking them down into simpler subproblems and storing their results to avoid redundant computations. It is applicable to problems with overlapping subproblems and optimal substructure. Unlike plain recursion, DP uses memoization or tabulation to improve efficiency, reducing time complexity significantly. For example, calculating Fibonacci numbers using recursion is exponential, while using DP makes it linear. DP is particularly useful in solving optimiza

In [ ]:
chat = [
    {"role": "user", "content": f"""
"Question": "What is time complexity and why is it important in algorithm analysis?",
"Rubric": "A correct definition of time complexity in terms of input size and execution steps (3 marks)\nAn explanation of how time complexity is expressed using Big-O notation (4 marks)\nA discussion of why time complexity matters in evaluating algorithm efficiency and scalability (3 marks)",
"Ideal Answer": "Time complexity refers to the computational complexity that describes the amount of time an algorithm takes to run as a function of the input size. It helps us estimate how the algorithm performs as input grows. It is typically expressed using Big-O notation, such as O(n), O(log n), or O(n^2), which gives an upper bound on runtime. Time complexity is crucial for analyzing and comparing algorithms, especially for large inputs, since it affects performance, scalability, and feasibility.",
"Student Answer": "Time complexity is about how long a program takes to run. We use O notation like O(n), but I don’t fully understand what it means. I think it’s important because we don’t want the program to be slow. It helps compare algorithms, but I sometimes forget the different types like linear or quadratic.",
"Strategy": "clarification"
Please evaluate the student's answer by assigning a score from 0 to 10 and providing a concise review.
Return only this format:
Score: <score from 0 to 10>
Review: <your feedback>
"""}
]

response = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(response, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=150, eos_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<|im_start|>user

"Question": "What is time complexity and why is it important in algorithm analysis?",
"Rubric": "A correct definition of time complexity in terms of input size and execution steps (3 marks)
An explanation of how time complexity is expressed using Big-O notation (4 marks)
A discussion of why time complexity matters in evaluating algorithm efficiency and scalability (3 marks)",
"Ideal Answer": "Time complexity refers to the computational complexity that describes the amount of time an algorithm takes to run as a function of the input size. It helps us estimate how the algorithm performs as input grows. It is typically expressed using Big-O notation, such as O(n), O(log n), or O(n^2), which gives an upper bound on runtime. Time complexity is crucial for analyzing and comparing algorithms, especially for large inputs, since it affects performance, scalability, and feasibility.",
"Student Answer": "Time complexity is about how long a program takes to run. We use O notation

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# Load your saved fine-tuned model and tokenizer
model_path = "/content/drive/MyDrive/fine-tuned-model1"  # or your Hugging Face path
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype=torch.float16)  # use float16 if it's a 4bit model

# Create the chat format (based on Hugging Face chat template compatibility)


In [ ]:
chat = [
    {"role": "user", "content": f"""
"Question": "What is the primary objective of requirements modeling, and how does it contribute to the software development process?",
    "Rubric": "A clear description of the primary objective of requirements modeling, including its role in describing customer requirements (3 marks)\nAn explanation of how requirements modeling establishes a basis for software design, including the importance of bridging the gap between system-level representation and software design (4 marks)\nA discussion of the role of requirements modeling in defining a set of requirements that can be validated once the software is built, including the significance of validation in the software development process (3 marks)",
    "Ideal Answer": "The primary objective of requirements modeling is to create a variety of representations that describe what the customer requires, establishing a basis for the creation of a software design, and defining a set of requirements that can be validated once the software is built. Requirements modeling plays a crucial role in bridging the gap between a system-level representation that describes overall system and business functionality and a software design. By doing so, it enables the development of a software design that meets the customer's requirements and ensures that the software functions as intended. Furthermore, requirements modeling defines a set of requirements that can be validated once the software is built, allowing for the verification of the software's functionality and performance. This validation is essential in ensuring that the software meets the customer's needs and is free from defects, ultimately contributing to the success of the software development process.",
    "Student Answer": "To be honest, I'm not entirely sure about the primary objective of requirements modeling, but from what I've gathered, it's basically about creating a bunch of diagrams and documents that describe what the customer wants. I think it's supposed to help us understand the customer's requirements, but I'm not really clear on how it does that. \n\nI've heard that requirements modeling is important for software design, but I don't really get how it bridges the gap between system-level representation and software design. I mean, isn't software design just about writing code? I'm not sure how requirements modeling fits into that. Maybe it's like, a way to make sure we're building the right thing, but I'm not really sure.\n\nAs for validation, I think requirements modeling is supposed to help us define a set of requirements that we can check against once the software is built. But, to be honest, I'm not really sure how that works or why it's important. I know validation is a thing, but I don't really understand its significance in the software development process. Can we come back to this later? I feel like I'm missing something. \n\nOh, and I think requirements modeling might also have something to do with, like, making sure the software is usable and stuff. But, yeah, that's about it. I'm pretty lost on this topic, sorry.",
    "Strategy": "clarification"
     Please evaluate the student's answer by assigning a score from 0 to 10 and providing a concise review.
Return only this format:
Score: <score from 0 to 10>
Review: <your feedback>
"""}

]

response = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(response, return_tensors="pt").to(model.device)
inputs = {k: v for k, v in inputs.items() if k in ['input_ids', 'attention_mask']} # Remove token_type_ids if present

outputs = model.generate(**inputs, max_new_tokens=150) # Corrected from model_path.generate
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

<|im_start|>user

"Question": "What is the primary objective of requirements modeling, and how does it contribute to the software development process?",
    "Rubric": "A clear description of the primary objective of requirements modeling, including its role in describing customer requirements (3 marks)
An explanation of how requirements modeling establishes a basis for software design, including the importance of bridging the gap between system-level representation and software design (4 marks)
A discussion of the role of requirements modeling in defining a set of requirements that can be validated once the software is built, including the significance of validation in the software development process (3 marks)",
    "Ideal Answer": "The primary objective of requirements modeling is to create a variety of representations that describe what the customer requires, establishing a basis for the creation of a software design, and defining a set of requirements that can be validated once the

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
